# More guidance
In this notebook, we're going to learn how even more about Guidance, a programming paradigm that offers superior control and efficiency compared to conventional prompting and chaining

In [2]:
!pip install guidance


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [218]:
import guidance
from guidance import models, gen, select, user, assistant, system, substring

In [246]:
# model_path = "models/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
model_path = "models/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf"
llm = models.LlamaCpp(model_path, n_gpu_layers=1)
llm_chat = models.LlamaCppChat(model_path, n_gpu_layers=1, n_ctx=2048)

ggml_metal_free: deallocating


In [238]:
@guidance
def experts(lm, query):
    with system():
        lm += "You are a helpful assistant."

    with user():
        lm += f"""\
        I want a response to the following question:
        {query}
        But don't answer that question just yet. Instead tell me 3 world-class experts (past or present) who would be great at answering this?"""

    with assistant():
        lm += gen(name='experts', max_tokens=300)
    
    with user():
        lm += f"""\
        Great, now please answer the question as if these experts had collaborated in writing a joint answer.        
        If the experts would disagree, present their different positions as alternatives and indicate who disagrees with who.
        Please start your answer with ANSWER:"""
    
    with assistant():
        lm += gen(name='answer', max_tokens=500)
    return lm

In [239]:
llm_chat + experts(query='What is a Large Language Model?')

In [237]:
llm_chat + experts(query='Who is the best female footballer of the last 20 years?')

In [240]:
llm_chat + experts(query='Has social media been a net positive or net negative to society?')

In [241]:
llm_chat + experts(query="What's the best graph database?")

In [247]:
llm_chat + experts(query="What is Kafka?")

In [251]:
for chunk in llm_chat + experts(query="What is Kafka?"):
  if chunk:
    print(chunk)

TypeError: 'async for' requires an object with __aiter__ method, got LlamaCppChat